<a href="https://colab.research.google.com/github/susumu2357/US-patent-analysis/blob/master/colab/Dataset_with_background_of_the_invention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!gsutil cp gs://mlstudy-phys/data/citations_info_3000+3000.df.gz ./citations_info_3000+3000.df.gz
!gsutil cp gs://mlstudy-phys/data/grants_for_3000+3000.df.gz ./grants_for_3000+3000.df.gz
!gsutil cp gs://mlstudy-phys/data/testset_app_3000.df.gz ./testset_app_3000.df.gz
!gsutil cp gs://mlstudy-phys/data/training_app_3000.df.gz ./training_app_3000.df.gz  

Copying gs://mlstudy-phys/data/citations_info_3000+3000.df.gz...
/ [1 files][506.5 KiB/506.5 KiB]                                                
Operation completed over 1 objects/506.5 KiB.                                    
Copying gs://mlstudy-phys/data/grants_for_3000+3000.df.gz...
\ [1 files][129.4 MiB/129.4 MiB]                                                
Operation completed over 1 objects/129.4 MiB.                                    
Copying gs://mlstudy-phys/data/testset_app_3000.df.gz...
- [1 files][ 45.5 MiB/ 45.5 MiB]                                                
Operation completed over 1 objects/45.5 MiB.                                     
Copying gs://mlstudy-phys/data/training_app_3000.df.gz...
- [1 files][ 45.0 MiB/ 45.0 MiB]                                                
Operation completed over 1 objects/45.0 MiB.                                     


In [0]:
import pandas as pd

citations_info = pd.read_pickle("./citations_info_3000+3000.df.gz")
grants = pd.read_pickle("./grants_for_3000+3000.df.gz")
testset_app_3000 = pd.read_pickle("./testset_app_3000.df.gz")
training_app_3000 = pd.read_pickle("./training_app_3000.df.gz")

In [0]:
training_app_3000["xml"][0]

'<us-patent-application lang="EN" dtd-version="v4.2 2006-08-23" file="US20090299431A1-20091203.XML" status="PRODUCTION" id="us-patent-application" country="US" date-produced="20091118" date-publ="20091203">\n<us-bibliographic-data-application lang="EN" country="US">\n<publication-reference>\n<document-id>\n<country>US</country>\n<doc-number>20090299431</doc-number>\n<kind>A1</kind>\n<date>20091203</date>\n</document-id>\n</publication-reference>\n<application-reference appl-type="utility">\n<document-id>\n<country>US</country>\n<doc-number>12130785</doc-number>\n<date>20080530</date>\n</document-id>\n</application-reference>\n<us-application-series-code>12</us-application-series-code>\n<classifications-ipcr>\n<classification-ipcr>\n<ipc-version-indicator><date>20060101</date></ipc-version-indicator>\n<classification-level>A</classification-level>\n<section>A</section>\n<class>61</class>\n<subclass>M</subclass>\n<main-group>1</main-group>\n<subgroup>08</subgroup>\n<symbol-position>F</sy

In [0]:
import re
BACKGROUND_TO_SUMMARY = re.compile(r'<\?summary-of-invention[^>]*>(.*)<\?summary-of-invention[^>]*>',re.MULTILINE|re.DOTALL)
BACKGROUND_TO_SUMMARY_GRANT = re.compile(r'<\?BRFSUM[^>]*>(.*)<\?BRFSUM[^>]*>',re.MULTILINE|re.DOTALL)

TAG_PAT = re.compile(r"<.*?>")
LB_PAT = re.compile(r'[\t\n\r\f\v][" "]*')

def whole_xml_to_background_xml(whole):
    mat = BACKGROUND_TO_SUMMARY.search(whole)
    if mat is None:
      return ""
    else:
      return mat.group(1)
  
def whole_xml_to_background(whole):
    return TAG_PAT.sub(' ', whole_xml_to_background_xml(whole))

def whole_xml_to_background_xml_grant(whole):
    mat = BACKGROUND_TO_SUMMARY_GRANT.search(whole)
    if mat is None:
      return ""
    else:
      return mat.group(1)
  
def whole_xml_to_background_grant(whole):
    return TAG_PAT.sub(' ', whole_xml_to_background_xml_grant(whole))

def remove_linebreak_from_claim(claim):
    '''
    Remove line break symbol "\n" with space(s).
    '''
    return LB_PAT.sub('', claim)

In [0]:
training_app_3000["background"] = training_app_3000["xml"].map(whole_xml_to_background).map(remove_linebreak_from_claim)

In [0]:
training_app_3000["background"][training_app_3000["background"] == ""].index

Int64Index([213], dtype='int64')

In [0]:
training_app_3000["background"][213] = "background of the invention"

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
training_app_3000["background"][training_app_3000["background"] == ""]

Series([], Name: background, dtype: object)

In [0]:
testset_app_3000["background"] = testset_app_3000["xml"].map(whole_xml_to_background).map(remove_linebreak_from_claim)

In [0]:
testset_app_3000["background"][testset_app_3000["background"] == ""].index

Int64Index([], dtype='int64')

In [0]:
testset_app_3000.head()

,app_id,xml,background
0,12211548,"<us-patent-application lang=""EN"" dtd-version=""...",FIELD OF THE INVENTION The present invention r...
1,12765236,"<us-patent-application lang=""EN"" dtd-version=""...",The field of the invention is that of dental c...
2,12827269,"<us-patent-application lang=""EN"" dtd-version=""...",TECHNICAL FIELD Embodiments of the present inv...
3,14436491,"<us-patent-application lang=""EN"" dtd-version=""...",BACKGROUND Cancer is thought to occur as a res...
4,13633425,"<us-patent-application lang=""EN"" dtd-version=""...",CROSS-REFERENCE TO RELATED APPLICATIONS The pr...


In [0]:
grants["background"] = grants["xml"].map(whole_xml_to_background_grant).map(remove_linebreak_from_claim)

In [0]:
grants["background"][grants["background"] == ""].index

Int64Index([2309], dtype='int64')

In [0]:
grants["background"][2309] = "background of the invention"

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
grants["background"][grants["background"] == ""]

Series([], Name: background, dtype: object)

In [0]:
grants.head()

,parsed,xml,background
0,6837113,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",CROSS-REFERENCE TO RELATED APPLICATIONS Not Ap...
1,6837224,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",BACKGROUND OF THE INVENTION 1. Field of the In...
2,6837699,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",BACKGROUND OF THE INVENTION 1) Field of the In...
3,6837886,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",BACKGROUND OF THE INVENTION 1. Field of the In...
4,6837901,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",FIELD OF THE INVENTION The present invention g...


In [0]:
import re
CLAIM_PAT = re.compile(r'<claims[^>]*>(.*)</claims>',re.MULTILINE|re.DOTALL)
TAG_PAT = re.compile(r"<.*?>")
LB_PAT = re.compile(r'[\t\n\r\f\v][" "]*')
CANCELED_PAT = re.compile(r'[0-9]+.*\. \(canceled\)[" "]')
NUM_PAT = re.compile(r'[" "]?[0-9]+[" "]?\.[" "]?')

def whole_xml_to_claim_xml(whole):
    mat = CLAIM_PAT.search(whole)
    return mat.group(1)
def whole_xml_to_claim(whole):
    return TAG_PAT.sub(' ', whole_xml_to_claim_xml(whole))

def remove_linebreak_from_claim(claim):
    '''
    Remove line break symbol "\n" with space(s).
    '''
    return LB_PAT.sub('', claim)

def remove_canceled_claim(claim):
    return CANCELED_PAT.sub('', claim)
  
def remove_claim_numbers(claim):
    return NUM_PAT.sub('', claim)  

In [0]:
training_app_3000["claim"] = training_app_3000["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)
testset_app_3000["claim"] = testset_app_3000["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)
grants["claim"] = grants["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)

In [0]:
training_app_3000 = training_app_3000.drop("xml", axis=1)
testset_app_3000 = testset_app_3000.drop("xml", axis=1)
grants = grants.drop("xml", axis=1)

In [0]:
training_app_3000.head()

,app_id,background,claim
0,12130785,FIELD OF THE INVENTION The present disclosure ...,A system for differentiating noise from an arr...
1,12652424,BACKGROUND An organization may store data in a...,A method of allocating resources in a data war...
2,12214532,The disclosure of Japanese Patent Application ...,A controlling method of a media processing app...
3,14204411,BACKGROUND OF THE INVENTION 1. Field of the In...,An electronic percussion instrument comprising...
4,13090122,FIELD OF THE INVENTION The present invention e...,A composition comprising at least one active a...


In [0]:
testset_app_3000.head()

,app_id,background,claim
0,12211548,FIELD OF THE INVENTION The present invention r...,A method for ranking an entity against a group...
1,12765236,The field of the invention is that of dental c...,A dental composition comprising:(1) at least o...
2,12827269,TECHNICAL FIELD Embodiments of the present inv...,"A vehicle, comprising:a chassis; a platform su..."
3,14436491,BACKGROUND Cancer is thought to occur as a res...,A conjugate comprising a GM-CSF polypeptide an...
4,13633425,CROSS-REFERENCE TO RELATED APPLICATIONS The pr...,"A method for manufacturing a stack package, co..."


In [0]:
grants.head()

,parsed,background,claim
0,6837113,CROSS-REFERENCE TO RELATED APPLICATIONS Not Ap...,A method for determining an estimated velocity...
1,6837224,BACKGROUND OF THE INVENTION 1. Field of the In...,An evaporated fuel treatment device for intern...
2,6837699,BACKGROUND OF THE INVENTION 1) Field of the In...,An apparatus for clamping used in molding appl...
3,6837886,BACKGROUND OF THE INVENTION 1. Field of the In...,"A system, comprising:a catheter having a braid..."
4,6837901,FIELD OF THE INVENTION The present invention g...,The method of delivering a shape memory stent ...


In [0]:
train_data = pd.merge(training_app_3000, citations_info, on='app_id')[['app_id', 'claim', 'background', 'parsed']]
test_data = pd.merge(testset_app_3000, citations_info, on='app_id')[['app_id', 'claim', 'background', 'parsed']]

In [0]:
train_data['label'] = 1
test_data['label'] = 1

In [0]:
train_data = train_data.merge(grants, how='inner', on='parsed')
test_data = test_data.merge(grants, how='inner', on='parsed')

In [0]:
train_data.head()

,app_id,claim_x,background_x,parsed,label,background_y,claim_y
0,12130785,A system for differentiating noise from an arr...,FIELD OF THE INVENTION The present disclosure ...,7212849,1,CROSS-REFERENCE TO RELATED APPLICATIONS This a...,"A cardiac rhythm management (CRM) system, comp..."
1,12652424,A method of allocating resources in a data war...,BACKGROUND An organization may store data in a...,7814459,1,TRADEMARKS IBM&#xae; is a registered trademark...,A computer-implemented method of automated dat...
2,12652424,A method of allocating resources in a data war...,BACKGROUND An organization may store data in a...,7814459,1,TRADEMARKS IBM&#xae; is a registered trademark...,A computer-implemented method of automated dat...
3,12214532,A controlling method of a media processing app...,The disclosure of Japanese Patent Application ...,7652956,1,"This application is a 371 of PCT/JP05/10057, f...",A disc drive comprising:a housing having forme...
4,14204411,An electronic percussion instrument comprising...,BACKGROUND OF THE INVENTION 1. Field of the In...,7439432,1,BACKGROUND OF THE INVENTION 1. Field of the In...,"A pad for an electronic drum, comprising:a bod..."


In [0]:
test_data.head()

,app_id,claim_x,background_x,parsed,label,background_y,claim_y
0,12211548,A method for ranking an entity against a group...,FIELD OF THE INVENTION The present invention r...,7092898,1,CROSS-REFERENCE TO SUBMITTED APPENDICIES The f...,A data processing method using a computer netw...
1,12765236,A dental composition comprising:(1) at least o...,The field of the invention is that of dental c...,7740482,1,BACKGROUND 1 Filed of the Invention The field ...,A dental composition comprising:(1) at least o...
2,12827269,"A vehicle, comprising:a chassis; a platform su...",TECHNICAL FIELD Embodiments of the present inv...,7213662,1,BACKGROUND OF THE INVENTION The present invent...,A mower having a pair of drive wheels operably...
3,14436491,A conjugate comprising a GM-CSF polypeptide an...,BACKGROUND Cancer is thought to occur as a res...,7217421,1,FIELD OF THE INVENTION The present invention r...,A composition for generating an immune respons...
4,13633425,"A method for manufacturing a stack package, co...",CROSS-REFERENCE TO RELATED APPLICATIONS The pr...,7989943,1,CROSS-REFERENCE TO RELATED APPLICATIONS The pr...,A stacked semiconductor package comprising:a s...


In [0]:
def pick_up_uncited_grants(df, app_id, n=1, random_state=23):
    '''
    Randomly pick up uncited grant pair to a given app_id for generating negative samples.
    '''
    n_rows = df[ df['app_id'] != app_id ].sample(n=n, random_state=random_state)
    
    return [n_rows['parsed'].values[0], 0 ,n_rows['claim_y'].values[0], n_rows['background_y'].values[0]]

In [0]:
seed = 23

train_non_cited_data = pd.DataFrame([
    [app_id, claimx, backgroundx] + pick_up_uncited_grants(train_data, app_id, random_state=seed+idx)
    for idx, (app_id, claimx, backgroundx)
    in enumerate(zip(train_data['app_id'], train_data['claim_x'], train_data['background_x']))
])

train_non_cited_data.columns = train_data.columns

train_non_cited_data.head()

,app_id,claim_x,background_x,parsed,label,background_y,claim_y
0,12130785,A system for differentiating noise from an arr...,FIELD OF THE INVENTION The present disclosure ...,7848141,0,A multi-level cell copyback program method in ...,CROSS-REFERENCE TO RELATED APPLICATIONS This a...
1,12652424,A method of allocating resources in a data war...,BACKGROUND An organization may store data in a...,7353145,0,A computer implemented method for correcting a...,CROSS REFERENCE TO RELATED APPLICATIONS This a...
2,12652424,A method of allocating resources in a data war...,BACKGROUND An organization may store data in a...,6928488,0,A serialization and deserialization architectu...,TECHNICAL FIELD The present invention relates ...
3,12214532,A controlling method of a media processing app...,The disclosure of Japanese Patent Application ...,7287122,0,A method of managing a cache structure of a mu...,BACKGROUND OF THE INVENTION 1. Field of the In...
4,14204411,An electronic percussion instrument comprising...,BACKGROUND OF THE INVENTION 1. Field of the In...,7058343,0,A non-marking media consumable material for us...,BACKGROUND 1. Field This invention is directed...


In [0]:
seed = 23

test_non_cited_data = pd.DataFrame([
    [app_id, claimx, backgroundx] + pick_up_uncited_grants(test_data, app_id, random_state=seed+idx)
    for idx, (app_id, claimx, backgroundx)
    in enumerate(zip(test_data['app_id'], test_data['claim_x'], test_data['background_x']))
])

test_non_cited_data.columns = test_data.columns

test_non_cited_data.head()

,app_id,claim_x,background_x,parsed,label,background_y,claim_y
0,12211548,A method for ranking an entity against a group...,FIELD OF THE INVENTION The present invention r...,6989640,0,An actuator driving device of a working machin...,TECHNICAL FIELD The present invention relates ...
1,12765236,A dental composition comprising:(1) at least o...,The field of the invention is that of dental c...,6969674,0,A method for making a fine pitch flip chip sub...,FIELD OF THE INVENTION The invention relates t...
2,12827269,"A vehicle, comprising:a chassis; a platform su...",TECHNICAL FIELD Embodiments of the present inv...,8012334,0,A composition comprising one or more sources o...,The present invention is directed to metal pla...
3,14436491,A conjugate comprising a GM-CSF polypeptide an...,BACKGROUND Cancer is thought to occur as a res...,7523553,0,A method of manufacturing an ink jet recording...,BACKGROUND OF THE INVENTION 1. Field of the In...
4,13633425,"A method for manufacturing a stack package, co...",CROSS-REFERENCE TO RELATED APPLICATIONS The pr...,6901791,0,A method for diagnosing a fuel supply system o...,FIELD OF THE INVENTION The present invention r...


In [0]:
train_data = pd.concat([train_data, train_non_cited_data]).reset_index(drop=True)
test_data = pd.concat([test_data, test_non_cited_data]).reset_index(drop=True)

train_data = train_data.loc[:, ['index', 'app_id', 'claim_x', 'background_x','parsed', 'claim_y', 'background_y', 'label']]
test_data = test_data.loc[:, ['index', 'app_id', 'claim_x', 'background_x', 'parsed', 'claim_y', 'background_y', 'label']]

train_data.columns = ['index', 'app_id', 'claim_app', 'background_app','parsed', 'claim_grant', 'background_grant', 'label']
test_data.columns = ['index', 'app_id', 'claim_app', 'background_app','parsed', 'claim_grant', 'background_grant', 'label']

In [0]:
train_data = train_data.drop("index", axis=1)
test_data = test_data.drop("index", axis=1)

In [0]:
train_data = train_data.sample(frac=1, random_state=seed).reset_index(drop=True)
test_data = test_data.sample(frac=1, random_state=seed).reset_index(drop=True)

In [0]:
train_data.head()

,app_id,claim_app,background_app,parsed,claim_grant,background_grant,label
0,15116396,Ringed tubular sheath comprising:two internal ...,The invention relates to a ringed tubular shea...,7706961,BACKGROUND OF THE INVENTION 1. Field of the In...,"A method of controlling a diesel engine, compr...",0
1,12312598,An exhaust purification system of an internal ...,TECHNICAL FIELD The present invention relates ...,7150145,A method for controlling dosage of an external...,This invention relates generally to nitrogen o...,1
2,13835416,A shell-type gyroscope comprising:a resonator ...,TECHNICAL FIELD The present invention relates ...,8150872,FIELD OF THE INVENTION The present invention r...,"A query system, comprising:a computing device ...",0
3,13709382,A bicycle rear derailleur comprising:a base me...,BACKGROUND 1. Field of the Invention This inve...,6997835,A derailleur for a bicycle comprising:a base m...,BACKGROUND OF THE INVENTION The present invent...,1
4,14927045,"A manufacturing method of a display device, co...",RELATED APPLICATIONS This application claims p...,7273403,A method of manufacturing flat display apparat...,BACKGROUND OF THE INVENTION 1. Field of the In...,1


In [0]:
import gzip
import pickle

filename = "train_data_with_background.pkl.gz"

with gzip.open(filename, 'w') as f:
  pickle.dump(train_data, f)
!gsutil cp {filename} gs://mlstudy-phys/data/

Copying file://train_data_with_background.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.8 MiB.                                     


In [0]:
import gzip
import pickle

filename = "test_data_with_background.pkl.gz"

with gzip.open(filename, 'w') as f:
  pickle.dump(test_data, f)
!gsutil cp {filename} gs://mlstudy-phys/data/

Copying file://test_data_with_background.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.8 MiB.                                     


In [0]:
training_app_3000.head()

,app_id,background,claim
0,12130785,FIELD OF THE INVENTION The present disclosure ...,A system for differentiating noise from an arr...
1,12652424,BACKGROUND An organization may store data in a...,A method of allocating resources in a data war...
2,12214532,The disclosure of Japanese Patent Application ...,A controlling method of a media processing app...
3,14204411,BACKGROUND OF THE INVENTION 1. Field of the In...,An electronic percussion instrument comprising...
4,13090122,FIELD OF THE INVENTION The present invention e...,A composition comprising at least one active a...


In [0]:
import gzip
import pickle

filename = "training_app_3000_with_background.pkl.gz"

with gzip.open(filename, 'w') as f:
  pickle.dump(training_app_3000, f)
!gsutil cp {filename} gs://mlstudy-phys/data/

Copying file://training_app_3000_with_background.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/8.5 MiB.                                      


In [0]:
testset_app_3000.head()

,app_id,background,claim
0,12211548,FIELD OF THE INVENTION The present invention r...,A method for ranking an entity against a group...
1,12765236,The field of the invention is that of dental c...,A dental composition comprising:(1) at least o...
2,12827269,TECHNICAL FIELD Embodiments of the present inv...,"A vehicle, comprising:a chassis; a platform su..."
3,14436491,BACKGROUND Cancer is thought to occur as a res...,A conjugate comprising a GM-CSF polypeptide an...
4,13633425,CROSS-REFERENCE TO RELATED APPLICATIONS The pr...,"A method for manufacturing a stack package, co..."


In [0]:
import gzip
import pickle

filename = "testset_app_3000_with_background.pkl.gz"

with gzip.open(filename, 'w') as f:
  pickle.dump(testset_app_3000, f)
!gsutil cp {filename} gs://mlstudy-phys/data/

Copying file://testset_app_3000_with_background.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  8.5 MiB/  8.5 MiB]                                                
Operation completed over 1 objects/8.5 MiB.                                      


In [0]:
grants.head()

,parsed,background,claim
0,6837113,CROSS-REFERENCE TO RELATED APPLICATIONS Not Ap...,A method for determining an estimated velocity...
1,6837224,BACKGROUND OF THE INVENTION 1. Field of the In...,An evaporated fuel treatment device for intern...
2,6837699,BACKGROUND OF THE INVENTION 1) Field of the In...,An apparatus for clamping used in molding appl...
3,6837886,BACKGROUND OF THE INVENTION 1. Field of the In...,"A system, comprising:a catheter having a braid..."
4,6837901,FIELD OF THE INVENTION The present invention g...,The method of delivering a shape memory stent ...


In [0]:
import gzip
import pickle

filename = "grants_for_3000+3000_with_background.pkl.gz"

with gzip.open(filename, 'w') as f:
  pickle.dump(grants, f)
!gsutil cp {filename} gs://mlstudy-phys/data/

Copying file://grants_for_3000+3000_with_background.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/23.8 MiB.                                     
